<a href="https://colab.research.google.com/github/ahammedrohit/Speech-Recognition-using-wav2vec2-with-minimum-GPU/blob/main/wav2vec2_with_chunk_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!nvidia-smi
!pip install -q transformers
!pip install -q pydub
!pip install -q torch 



Python 3.7.12
Tue Jan 18 05:34:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |   2984MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                          

In [ ]:
from IPython.display import YouTubeVideo
import librosa
import soundfile as sf
!apt-get install -qq sox
!pip install -q youtube-dl
!pip3 install -q pydub
!apt-get install -y -qq ffmpeg
#Calculate WER
!pip install -q jiwer

In [ ]:
import nltk
import librosa
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# importing libraries 
#from speechbrain.pretrained import EncoderDecoderASR
#import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

import torch
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [ ]:
def correct_uppercase_sentence(input_text): 
  """
  Returns the corrected sentence
  """  
  sentences = nltk.sent_tokenize(input_text)
  return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

# Large file transcription using chunks with "facebook/wav2vec2-large-960h-lv60-self" model

In [ ]:
### Youtube Vdeos ###


#YOUTUBE_ID = 't2qlZHW-fDI'
#YOUTUBE_ID = '-yshHFseMyI'

#Representing integers
#YOUTUBE_ID = 'CAOJPoSzJA0'

#In this tutorial
#YOUTUBE_ID = 'yiKryjS1Cec'

#Hi I'm Ray Jordan
YOUTUBE_ID = 'Ux4brKS0Sac'

#The Joe Rogan
#YOUTUBE_ID = 'Sv4jq8t-7us'

#Deepika
#YOUTUBE_ID = 'CD67bwUQgtE'

#When I was 27 years
#YOUTUBE_ID = 'H14bBuluwB8'

!rm -rf *.wav
!youtube-dl --extract-audio --audio-format wav --output "test.%(ext)s" https://www.youtube.com/watch\?v\={YOUTUBE_ID}


#Sampling to 16000Hz
print("Changing Sampling rate to 16000kHz")
speech, rate = librosa.load("test.wav",sr=16000)
print("Successfully sampled to ", rate,"kHz")

[youtube] Ux4brKS0Sac: Downloading webpage
[download] Destination: test.m4a
[download] 100% of 1.84MiB in 00:21
[ffmpeg] Correcting container in "test.m4a"
[ffmpeg] Destination: test.wav
Deleting original file test.m4a (pass -k to keep)
Changing Sampling rate to 16000kHz
Successfully sampled to  16000 kHz


In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_ID = "facebook/wav2vec2-large-960h-lv60-self"

wav_input = 'test.wav'


tokenizer = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
model.to(device)


#asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="pretrained_models/asr-crdnn-rnnlm-librispeech",  run_opts={"device":"cuda"})
#asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-transformerlm-librispeech", savedir="pretrained_models/asr-crdnn-transformerlm-librispeech",  run_opts={"device":"cuda"})
#asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-transformer-transformerlm-librispeech", savedir="pretrained_models/asr-transformer-transformerlm-librispeech",  run_opts={"device":"cuda"})

# create a speech recognition object
#r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")

        # recognize the chunk

        #text = asr_model.transcribe_file(chunk_filename)
        #whole_text += text

        try:
          #text = asr_model.transcribe_file(chunk_filename)
          #read the file
          speech, samplerate = sf.read(chunk_filename)
          #make it 1-D
          if len(speech.shape) > 1: 
            speech = speech[:,0] + speech[:,1]
          #Resample to 16khz
          if samplerate != 16000:
            speech = librosa.resample(speech, samplerate, 16000)
            #tokenize
          input_values = tokenizer(speech, return_tensors="pt").input_values
          #take logits
          with torch.no_grad():
            logits = model(input_values.to(device)).logits
            
          #take argmax (find most probable word id)
          predicted_ids = torch.argmax(logits, dim=-1)
          #get the words from the predicted word ids
          transcription = tokenizer.decode(predicted_ids[0])
          #output is all uppercase, make only the first letter in first word capitalized
          transcription = correct_uppercase_sentence(transcription.lower())


        except:
          a = 0
        else:
          transcription = f"{transcription.capitalize()}. "
          whole_text += transcription
    # return the text for all chunks detected
    return whole_text

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/162 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
path = "test.wav"


output_file = "output.txt"
if not os.path.exists(output_file):
    os.mknod(output_file)

with open(output_file, 'w') as f:
    f.write(get_large_audio_transcription(path))
    print("Text Saved in output.txt")
    
f = open(output_file, "r")
print(f.readline())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Text Saved in output.txt
I i'm ray jordan with chrismus and company to day o my home ideas dot com we're gong to be wrapping packages. The first one is thi small square package and i picked this great turquoise paper with the red cardi was to start. First of all make sure your paper covers your package. So this one's a little bit longer. So whenhe tojust. . Cut off the additiona wer not going to use. On the sides of your package make sure it's a little bit less than the height of your box so we're gong to trim up. Then. From that with correct get the length on your paper correct. . And trim that up. D. Now o'you got the right size of paper. . Pull over the box. Tape. It always looks best to seam the paper too so to make sure of t gt a nice square. Pull the paper over take the other side. And when you're going down on the sides. Pristol. Do a nice crease on the sides. Said look sharper. And then fold in. Again creasing the paper as you go. To make sure you have a nice. Clean finish. And

In [ ]:
#Calculate WER
!pip install -q jiwer
from jiwer import wer
from jiwer import cer

#WER of "speechbrain/asr-transformer-transformerlm-librispeech" model
#transcript = "I am speaking to you from the Cabinet Room of ten Downing Street. This morning the British Ambassador in Berlin handed the German government a final note, stating that unless we heard from them by eleven o'clock that they were prepared at once to withdraw their troops from Poland, a state of war would exist between us. i have to tell you now that no such undertaking has been received and that consequently this country is at war with Germany."
#transcript = "Representing integers in Python. Firstly a quick refresher. What is an integer? An integer is a whole number. By whole number we mean it has no fractional or decimal component. An integer can be stored using different types. Two possible Python datatypes representing an integer are firstly string secondly an int. let's take a look at those. Firstly you can represent an integer using a string literal using the notation shown below. You can see it's been assigned with 110 in quotes. this could be single or double quotes. here Python understands you want to store the integer 110 as a string. you can do the same with integer data type as seen here. notice there are no quotes around the number. it's important to consider what you specifically mean by a hundred and ten in quotes and 110 without the quotes in the examples above. as a human who's used the decimal number system for your whole life it might be obvious to you that you mean the number 110. however there are several other number systems such as binary and hexadecimal which use different bases to represent an integer. let's take a look at that base system using the Python repple. here I'm going to be using bPython as it color codes the output but the standard Python wrapper will work in exactly the same way. so we've assumed that the numbers 1 1 0 mean decimal 110. but in Python we can prefix this number with different codes to tell python to use different number bases. so by typing 0 B before our number we can see this actually represents six in binary we have one four one two and no units. doing the same in octal with a no leads us to the value 72 as we have 164 1 8 and again no units. doing the same in hexadecimal with an X gives us the value 272. here we have one 256 a 16 and once more no units. if there's no prefix present int will assume that the number will be decimal. placing the prefix allows in to understand the number that we want. however if we don't use the prefix will a syntax error. string representations do not suffer from this you because strings hold arbitrary text data. now that you have some foundational knowledge about how to represent integers using string and in data types you'll learn how to convert a Python string to an INT."
#transcript = "In this tutorial we will teach you how to move an object on a photo using Adobe Photoshop. First of all open the image on which you want to make changes. Once done double click on it in the layers panel to convert it to layer 0 from background layer. After this make a selection around the object which needs to be moved using the polygonal lasso tool from the toolbar on the left. Now press ctrl X to cut the selection area and create a new layer from layers panel. Press ctrl V to paste the object on the new layer. Adjust the position of the object and use the eraser tool to rub off its edges so that it merges with the background. Next go back to layer 0 there will be a patch missing from the image. Make a selection around the patch using the polygonal lasso tool. Now go to the Edit menu at the top and select fill from this list of options. When the dialog box appears set the use to content-aware mode to normal opacity to 100% and press ok. Photoshop will now automatically fill the missing patch. If the patch is too large apply this option in 2 or 3 parts. You can then use the smudge tool to even out the area. In the end select both layers and press ctrl e to merge them."
transcript = "Hi I'm ray Jordan with Christmas accompany. Today on my home ideas comm we're going to be wrapping packages. First one it's a small square package and I picked this great turquoise paper with the red Cardinals to start. First of all make sure your paper covers your package. So this one's a little bit longer. So we need to just cut off the additional we're not going to use. On the sides of your package make sure it's a little bit less than the height of your box. So we're going to trim up. Then from that with correct get the length on your paper correct. And trim that up. Now you have the right size of paper pull over the box tape. Always looks best to seam the paper to so to make sure to get you a nice square. Pull the paper over tape the other side. Then when you're going down on the sides. Press down. Do a nice crease on the sides so it looks sharper. And then fold in again creasing the paper as you go to make sure you have a nice clean finish. And pull the paper tab up get your tape tape there but you've got a nice clean square. The other side doing the same thing press down your paper. Seam press in the sides do your seam pull up and tape. Then again just kind of seemed press down."
#transcript = "The Joe Rogan experience. This season it was a lot of it was some baby steps of like going back to that route and just placing a lot of gears so i felt super safe and then you know climbing it in the same style that i was going to climb with Alex Simon climbing it but still placing more gear and then during my try this year i actually told Alex because he was he's definitely um more comfortable than almost anyone I know on el cap most people at least feel a little bit of intimidation a little bit of fear like you know there's a little bit of anxiety around it and he's just it's so casual that he was like yeah you should just like you know just run it out like go go all the way like he was basically just wanting me to do it the same way that i was doing it before and I was like you know Alex I’m gonna be a little slower this time like I just I think I need that for my head and so he felt like you should do it with the same amount of gear that you did before just don't fall just don't fall this time Emily yeah kinda real simple a little bit yeah he was like you know but Alex the one thing he always says is he's just like okay follow your heart and i was like okay well I’m gonna go slower. Well just so the audience knows Alex told me that he did a route once and, on his way up realized that he hadn't brought any chalk. So, he had to borrow it from someone else he was strapped to ropes so he's free soloing yeah, no chalk yeah says hey uh i don't have any chalk and the guy gives him a bag and said he left the bag at the top so that when they got to the top, they could retrieve the bag yeah that's a classic Alex situation yeah what like how do you not bring like if you lift weights, you like chalk it's like it's very important for me to get a good grip kind of obsessed with having chalk I don't have chalk I just won't go climbing i just can't imagine but you know I think he's the type of person that um you know going back to like talking about heart rate like I bet I bet when he's in those types of situations like he's so relaxed and his heart rate is so low and he's just he's a different I  think he's truly unique you know and i i think i i think that he just has a different like even brain chemistry than than a lot of us in terms of like how he feels fear and how he can maintain that composure in a very dangerous situation. Well he's remarkably calm all the time yeah like he just seems always like on this one plane yeah he's got like this like 55 beats per minute that he just stays at all the time yep and it's great climbing with him actually because it kind of rubs off on me a little bit like when i climb with Alex i feel more confident I feel more capable I tend to climb better um even though he kind of gives me all the time for being nervous he's always like you're always nervous like you're you're always stressing out but it's actually when I’m with him I’m like you know what i i can chill out a little bit i can like i can i can be on that wavelength a little bit more so that's part of the reason why i like climbing with him up there and why I chose him for this project well that makes sense i mean i guess when you're around people that are if you're doing a thing and you're around people that are excellent at that thing it's contagious or at least inspirational yeah i mean i think it's important to like do yeah to practice practice things you want to be better at with people who are better than you yeah for sure episodes of the joe Rogan experience are now free on Spotify that's right they're free from September 1st to December 1st they're going to be available everywhere but after December 1st they will only be available on Spotify but they will  l be free that includes the video the video will also be there it'll also be free that's all we're asking just go download Spotify much love."
#transcript = "Welcome, this is your first American talk show.  Yes.  So, thank you for choosing us to be your first.  Thank you,  thank you.  So this is, first of all, before we talk about the movie and how you got into this business, your dad was a professional badminton player, right?  Yes.  I love badminton so much. And so, it's indoor, right?  Yes.  Really fast.  Yeah.  So, he was a professional player, and then, did you play as well?  I did, and he played at a time when, in India, nobody really played professional badminton. Everyone became a lawyer or a doctor or an engineer. And he just went ahead and did what he wanted to do. Yeah, and did he want you to do that? Did he want you to follow in his footsteps?  I think that would make him happy, but at the same time, he always said follow your heart. And I think when I was about 16, I always knew that being in front of the camera is something that I enjoy doing a lot more. And I sat down, I spoke to them one day, and I said this is what I want to do, I wanna be an actor, and they were very, very supportive. And I remember sitting them down one day, I told my mother. I said, can I please borrow 10,000 rupees from you, because I needed money for a portfolio for professional photographs to be taken and to send them around. And I told them that one day, I'll make them proud, and I'll repay them, pay that money back.Yeah, and I bet you have cuz you've done like 30 movies, right?  Yeah, something like that.  Yes, you have.You probably were gonna say 31.   I haven't counted actually,I should go back and do that.  Yeah, I think I read today that you did 30 movies in India, and so this is your first filmed here with Vin Diesel. That's a big deal to get into a movie with Vin Diesel, especially something like this. And how did you get this part?   I actually auditioned for Fast 7 about two or three years ago. And for whatever reason, that didn't work out. And I think people remembered me from that audition. Everyone talks about that chemistry, and people remember me from that. And then, about a year ago, I get a call from Paramount saying they're casting for this film. And I flew down, and I met DJ, my director, Caruso. And then, Vin found out in LA that I was in Toronto, and so overnight, luckily I had a visa, and he said I want her to come and see me. So overnight, I flew down from Toronto to Los Angeles to see them. And suddenly, in the middle of the night,we're doing this 45 minute photoshoot, professional with lights and this and that, and I was like, what is going on? Because I didn't know if I got the part. I still don't know if I got the part.We got into this really serious photoshoot. I don't know if you guys have seen the picture. And that was it.   That picture right there?  That's the one. That's in his house, yeah. That's the one, and he just put it up. I said I do hope you know that if you put that picture up, it means I'm doing this movie. He still didn't say anything.He just put it up.  And so, when that went up, you got all these calls from your fans and friends congratulating you. Yes, and I didn't know what to say to them because I had journalists calling me from India and saying, my God, you're doing this movie, and I didn't know what to say. And I just, I was like, I don't even know if that's Vin, I don't know what you guys are talking about.But I mean, or they're looking at it,going there's a romance going on, that's what you would get from that as well, which they're, look at that face.  Does that mean there is? Cuz that's what the rumor is.   Well, there's no smoke without fire, and  look at that. So? But. But it's all in my head.   You know?   It's in his head right there.  You're putting stuff in his head right there. So yeah, I mean, in my head, I think like yeah, we're together and we have this amazing chemistry, and we live together and we have these amazing babies, but it's all in my head.   Now you just became crazy, I don't know what happened. It first was a crush and now you have babies, I don't know.   Never the crush.  All right, so, we won't know until next time you're here?   You should ask him that.Okay.   You should ask him.   I'll do that.   What he thinks.   What is his number?  911. All right, well you're delightful. Thank you so much for being here. It's XXX: Return of Xander Cage in theaters and IMAX this Friday. We'll be right back."
#transcript = "When I was 27 years old,I left a very demanding job in management consulting for a job that was even more demanding teaching. I went to teach seventh graders math in the New York City public schools. And like any teacher I made quizzes and tests.I gave out homework assignments. When the work came back .I calculated grades. What struck me was that IQ was not the only difference between my best and my worst students. Some of my strongest performers did not have stratospheric IQ scores. Some of my smartest kids weren't doing so well. And that got me thinking. The kinds of things you need to learn in seventh grade math, sure, they're hard: ratios, decimals, the area of a parallelogram. But these concepts are not impossible, and I was firmly convinced that every one of my students could learn the material if they worked hard and long enough. After several more years of teaching, I came to the conclusion that what we need in education is a much better understanding of students and learning from a motivational perspective, from a psychological perspective. In education, the one thing we know how to measure best is IQ. But what if doing well in school and in life depends on much more than your ability to learn quickly and easily? So I left the classroom, and I went to graduate school to become a psychologist. I started studying kids and adults in all kinds of super challenging settings, and in every study my question was, who is successful here and why? My research team and I went to West Point Military Academy. We tried to predict which cadets would stay in military training and which would drop out. We went to the National Spelling Bee and tried to predict which children would advance farthest in competition. We studied rookie teachers working in really tough neighborhoods, asking which teachers are still going to be here in teaching by the end of the school year, and of those, who will be the most effective at improving learning outcomes for their students? We partnered with private companies, asking, which of these salespeople is going to keep their jobs? And who's going to earn the most money? In all those very different contexts,one characteristic emerged as a significant predictor of success. And it wasn't social intelligence. It wasn't good looks, physical health, and it wasn't IQ. It was grit. Grit is passion and perseverance for very long-term goals. Grit is having stamina. Grit is sticking with your future, day in, day out, not just for the week, not just for the month, but for years, and working really hard to make that future a reality. Grit is living life like it's a marathon, not a sprint. A few years ago, I started studying grit in the Chicago public schools. I asked thousands of high school juniors to take grit questionnaires, and then waited around more than a year to see who would graduate. Turns out that grittier kids were significantly more likely to graduate, even when I matched them on every characteristic I could measure, things like family income, standardized achievement test scores, even how safe kids felt when they were at school. So it's not just at West Point or the National Spelling Bee that grit matters. It's also in school, especially for kids at risk for dropping out. To me, the most shocking thing about grit is how little we know, how little science knows, about building it. Every day parents and teachers ask me, How do I build grit in kids? What do I do to teach kids a solid work ethic? How do I keep them motivated for the long run? The honest answer is, I don't know. What I do know is that talent doesn't make you gritty. Our data show very clearly that there are many talented individuals who simply do not follow through on their commitments. In fact, in our data, grit is usually unrelated or even inversely related to measures of talent. So far, the best idea I've heard about building grit in kids is something called growth mindset. This is an idea developed at Stanford University by Carol Dweck, and it is the belief that the ability to learn is not fixed, that it can change with your effort. Dr. Dweck has shown that when kids read and learn about the brain and how it changes and grows in response to challenge, they're much more likely to persevere when they fail, because they don't believe that failure is a permanent condition. So growth mindset is a great idea for building grit. But we need more. And that's where I'm going to end my remarks, because that's where we are. That's the work that stands before us. We need to take our best ideas, our strongest intuitions, and we need to test them. We need to measure whether we've been successful, and we have to be willing to fail, to be wrong, to start over again with lessons learned. In other words, we need to be gritty about getting our kids grittier.Thank you."

with open("output.txt","r") as f:
  transcripted = f.readline()
  print("WER:", wer(transcripted, transcript))
  print("CER:", cer(transcripted, transcript))

WER: 0.35384615384615387
CER: 0.15283165244375485
